<a href="https://colab.research.google.com/github/DALTOKI99/MachineLearning/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EA%B0%9C%EC%9A%94_%EC%9E%90%EC%97%B0%EC%96%B4%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%98%88%EC%A0%9C(%EB%B9%88%EC%B9%B8%EC%98%88%EC%B8%A1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BERT로 빈칸에 들어갈 토큰 예측
- multi-lingual BERT 사용

In [ ]:
!pip install tensorflow_text

import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np

tf.__version__

In [ ]:
# 문장 -> BERT 모델의 입력형태로 변환

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
encoder_inputs = preprocessor(text_input)
preprocessed_text = tf.keras.Model(text_input, encoder_inputs)

- 단어 리스트 가져오기

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
vocab_list = []
vocab_path = '/gdrive/MyDrive/Lecturer/W6/data/multi-bert-cased-vocab.txt' # vocab 위치

with open(vocab_path) as f:
    while True:
        vocab = f.readline()
        if vocab == '':
            break
        vocab_list.append(vocab.replace('\n',''))
len(vocab_list)

119547

In [ ]:
vocab_list[50000:50000+10]

['héros',
 'особено',
 'Championnats',
 '1576',
 'occitana',
 'spoke',
 '##ạo',
 'exclusive',
 'Републике',
 'ফুট']

- BERT (MLM: Masked Languge Model) 모델 불러오기
> 빈칸에 나올 단어를 예측하는 모델

In [ ]:
# Multilingual BERT 모델 위치: https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4
seq_length = 128 
num_predict = 20

mlm_inputs = dict(
    input_word_ids=tf.keras.layers.Input(shape=(seq_length,), dtype=tf.int32),
    input_mask=tf.keras.layers.Input(shape=(seq_length,), dtype=tf.int32),
    input_type_ids=tf.keras.layers.Input(shape=(seq_length,), dtype=tf.int32),
    masked_lm_positions=tf.keras.layers.Input(shape=(num_predict,), dtype=tf.int32),
)

encoder = hub.load("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4")
mlm = hub.KerasLayer(encoder.mlm, trainable=False)
mlm_outputs = mlm(mlm_inputs)
mlm_logits = mlm_outputs["mlm_logits"]  # [batch_size, num_predict, vocab_size]
mlm_model = tf.keras.Model(mlm_inputs, mlm_logits)
# mlm_model.summary()

In [ ]:
FIRST_INPUT_INSTANCE = None
LAST_INPUT_INSTANCE = None

def predict_mlm(sentence, topn=5):
    global FIRST_INPUT_INSTANCE, LAST_INPUT_INSTANCE
    sentence_tensor = tf.constant([sentence])
    input_tensors = preprocessed_text(sentence_tensor)
    FIRST_INPUT_INSTANCE = input_tensors.copy()

    masked_position = np.argwhere(input_tensors['input_word_ids'].numpy()[0] == 168)[0][0] 
    input_masked_lm_positions = tf.constant([[masked_position]])

    bool_list = [[True]* 128]
    bool_list[0][masked_position] = False
    y = [[103] * 128]

    mask_next_idx = input_tensors['input_word_ids'].numpy()[0][masked_position+1]
    if mask_next_idx in [9633, 9233]: # 을, 를
        y[0][masked_position+1] = {9633: 10622, 9233: 11513}[mask_next_idx]
        bool_list[0][masked_position+1] = False
    input_tensors['input_word_ids'] = tf.where(bool_list, input_tensors['input_word_ids'], y)

    input_instance = dict(
        input_word_ids=input_tensors['input_word_ids'],
        input_mask=input_tensors['input_mask'],
        input_type_ids=input_tensors['input_type_ids'],
        masked_lm_positions=input_masked_lm_positions
    )

    LAST_INPUT_INSTANCE = input_instance
    output = mlm_model(input_instance)
    # print("output.shape:", output.shape)
    output_np = tf.nn.softmax(output).numpy()[0][0]

    return np.array(vocab_list)[np.argsort(-1 * output_np)][:topn].tolist()

In [ ]:
vocab_list[9633]

'을'

In [ ]:
vocab_list[9233]

'를'

In [ ]:
vocab_list[10622]

'##을'

In [ ]:
vocab_list[11513]

'##를'

In [ ]:
predict_mlm('나는 _을 하는 것을 좋아해요.')

['말', '음악', '술', '가', '마']

In [ ]:
predict_mlm('운동 선수는 _를 잘 합니다.')

['축구', '선수', '야구', '경기', '영어']

In [ ]:
predict_mlm('축구 선수는 _를 잘 합니다.')

['축구', '선수', '경기', '야구', '영어']

In [ ]:
predict_mlm('_ is a city in England.')

['London', 'Cheshire', 'England', 'Oxford', 'Birmingham']

In [ ]:
predict_mlm('_ is a programming language')

['Pascal', 'Python', 'programming', 'language', 'C']

In [ ]:
def run_bert_mlm():
    print('==================== BERT로 빈칸에 올 단어 예측 ====================')
    while True:
        sentence = input('빈칸과 함께 문장을 입력해주세요: ')
        if sentence == '0':
            break
        pred = predict_mlm(sentence)

        for i, token in enumerate(pred):
            print('{}위: {}'.format(i+1, token))
        print('==============================================================')

In [ ]:
run_bert_mlm()

==================== BERT로 빈칸에 올 단어 예측 ====================
빈칸과 함께 문장을 입력해주세요: _ is a city in South Korea.
1위: Seoul
2위: .
3위: ,
4위: -
5위: Korea
빈칸과 함께 문장을 입력해주세요: Seoul is a _ in South Korea.
1위: city
2위: state
3위: capital
4위: port
5위: town
빈칸과 함께 문장을 입력해주세요: 운동 선수는 _을 잘 합니다.
1위: 술
2위: 몸
3위: 힘
4위: 손
5위: 목
빈칸과 함께 문장을 입력해주세요: 운동 선수는 _를 잘 합니다.
1위: 축구
2위: 선수
3위: 야구
4위: 경기
5위: 영어
빈칸과 함께 문장을 입력해주세요: _ is a programming language.
1위: Python
2위: PHP
3위: C
4위: language
5위: Java
빈칸과 함께 문장을 입력해주세요: 我喜欢吃_。
1위: 吃
2위: 饭
3위: 人
4위: 了
5위: 的


KeyboardInterrupt: ignored